In [1]:
%load_ext autoreload
%autoreload

In [2]:
import sys
sys.path.append("../")

In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import os
from datetime import datetime
import matplotlib.pyplot as plt
from src.evaluation import *
from src.utils import load_fold, assets, non_train_cols, load_gresearch_raw
from src.features import engineer_all_features
from src.models import BaseCryptoLearner
plt.rcParams["figure.figsize"] = (16,9)

In [5]:
from scipy import stats
import shap
import lightgbm as lgb

In [6]:
supplemental_train = pd.read_csv('../data/gresearch/raw/supplemental_train.csv')

In [7]:
example_test = pd.read_csv('../data/gresearch/raw/example_test.csv')


In [8]:
asset_info = pd.read_csv('../data/gresearch/raw/asset_details.csv')

In [9]:
bst = lgb.Booster(model_file='../models/lgbmBaseline.txt')

# Fast Feature Calculation 

- Goal is to have a very fast feature calculation on the fly in the prediction loop
- Should also be usable in the training routine
- Row wise feature creation
- has to be able to create all relevant features on the fly 

In [10]:
# OHLCV Features
def fast_ohlcv_features(df, out_features):
    
    out_features[0] = df['Count']
    out_features[1] = df['Open']
    out_features[2] = df['High']
    out_features[3] = df['Low']
    out_features[4] = df['Close']
    out_features[5] = df['Volume']
    out_features[6] = df['VWAP']
    out_features[7] = (df.Close - df.Open) / df.Open
    out_features[8] = (np.log(df.Close / df.Open))
    out_features[9] = (df.High / df.Low)
    
    return out_features

# Relative change features
def fast_relative_features(feature_array, columns, period, out_features):
    
    for j in columns:
        
        value = np.log(feature_array[0, j] /
                       feature_array[-period, j])
        i = np.min(np.where(np.isnan(current_features)))
        out_features[i]= value
    
    return out_features

def fast_lagged_features(feature_array, columns, period, out_features):
    
    for j in columns:
        value = feature_array[-period, j]
        i = np.min(np.where(np.isnan(current_features)))
        out_features[i]= value
        
    return out_features

In [25]:
%%time
pred_time = supplemental_train.timestamp.max()
test_df = supplemental_train[supplemental_train.timestamp==pred_time]
data = supplemental_train[supplemental_train.timestamp<pred_time]
# Constants
max_lag = 5
max_relative = 60
max_lookback_minutes = max_relative + max_lag

max_lookback = data.timestamp.max() - (max_lookback_minutes * 60)
current_data = data[data.timestamp>=max_lookback]
print(current_data.shape)


# Initialize the feature array using pandas
features = []
for asset_id in assets:
    asset = current_data[current_data.Asset_ID == asset_id]
    asset_features = engineer_all_features(asset).iloc[:,2:]
    features.append(asset_features.values)
features = np.array(features)

(924, 10)
CPU times: user 350 ms, sys: 50.9 ms, total: 401 ms
Wall time: 400 ms


In [154]:

# From engineer_all_features
relative_cols = ['Count', 'Open',
                 'High', 'Low', 'Close',
                 'Volume', 'VWAP']

lagged_cols = ['direct_return', 'log_return', 'high_low_ratio',
                   'log_change_Count_1min', 'log_change_Open_1min',
                   'log_change_High_1min', 'log_change_Low_1min',
                   'log_change_Close_1min',
                   'log_change_Volume_1min', 'log_change_VWAP_1min']


relative_periods = [1, 60]
lagged_periods = [1, 2, 3, 4, 5]



To DO: Keep track of the timestamps and backfill

In [190]:
%%time
new_features = []
predictions = []
for asset_id in assets:
    
    asset = test_df[test_df.Asset_ID == asset_id].iloc[0]

    asset_features = features[asset_id]

    current_features = np.repeat(np.nan, repeats=features.shape[2])

    current_features = fast_ohlcv_features(asset, current_features)

    # Relative change features
    for period in relative_periods:
        current_features = fast_relative_features(asset_features,
                                                  columns=np.where(np.isin(feature_names,relative_cols))[0],
                                                  period=period,
                                                  out_features=current_features)

    # Lagged features
    for period in lagged_periods:
        current_features = fast_lagged_features(asset_features,
                                                np.where(np.isin(feature_names,lagged_cols))[0],
                                                period=period,
                                                out_features=current_features)
    # Predict
    current_features = np.array(current_features)
    
    # Append data
    predictions.append(bst.predict(np.array(current_features).reshape(1,-1))[0])
    new_features.append(current_features)

new_features = np.array(new_features)

# Append to old array and shift
features[:,-1,:] = new_features
features = features[:,1:,:]

# Make prediction dataframe

CPU times: user 87.7 ms, sys: 32.2 ms, total: 120 ms
Wall time: 47.1 ms


In [191]:
%%time
for asset_id in assets:
    engineer_all_features(current_data)

CPU times: user 303 ms, sys: 3.92 ms, total: 307 ms
Wall time: 306 ms


# Fitting

In [ ]:
# Load training data 
train_dir = '../data/gresearch/processed/fold_6/train/'
test_dir = '../data/gresearch/processed/fold_6/test/'

In [ ]:
train_df = load_fold(train_dir)
test_df = load_fold(test_dir)

In [ ]:
feature_cols = np.array([col for col in train_df.columns[4:]])
target_col = ['Target']

In [ ]:
lgb_train_data = lgb.Dataset(train_df[feature_cols],
                             label=train_df[target_col])

In [ ]:
param = {'objective': 'regression', 'metric': 'mae'}
num_round = 20

bst = lgb.train(param, lgb_train_data, num_round)

In [ ]:
# Save model
bst.save_model('../models/lgbmBaseline.txt')

In [ ]:
feature_cols = np.array([col for col in train_df.columns[4:]])
target_col = ['Target']

In [ ]:
feature_cols

In [ ]:
lgb_train_data = lgb.Dataset(train_df[feature_cols],
                             label=train_df[target_col])

In [ ]:
param = {'objective': 'regression', 'metric': 'mae'}
num_round = 20

bst = lgb.train(param, lgb_train_data, num_round)

In [ ]:
# Save model
bst.save_model('../models/lgbmBaseline.txt')

In [ ]:
pred = pd.DataFrame({'Asset_ID': train_df.Asset_ID,
                     'timestamp': train_df.timestamp})

pred['prediction'] = bst.predict(train_df[feature_cols])


test = train_df[['Target', 'Weight', 'timestamp', 'Asset_ID']].merge(
    pred, on=['timestamp', 'Asset_ID'], how='left').fillna(0)

score = corr_score(test.Target, test.prediction, test.Weight)
print(score)

In [ ]:
sns.scatterplot(test.Target, test.prediction)

In [ ]:
# Out of sample
pred = pd.DataFrame({'Asset_ID': test_df.Asset_ID,
                     'timestamp': test_df.timestamp})

pred['prediction'] = bst.predict(test_df[feature_cols])


test = test_df[['Target', 'Weight', 'timestamp', 'Asset_ID']].merge(
    pred, on=['timestamp', 'Asset_ID'], how='left').fillna(0)

score = corr_score(test.Target, test.prediction, test.Weight)
print(score)

In [ ]:
stats.pearsonr(test.Target,test.prediction)

In [ ]:
sns.scatterplot(test.Target, test.prediction)

In [ ]:
np.corrcoef((test.Target,test.prediction))

In [ ]:
shap_values = shap.TreeExplainer(bst).shap_values(train_df[feature_cols])

In [ ]:
shap.summary_plot(shap_values, train_df[feature_cols])

In [ ]:
sns.scatterplot(x='log_change_High_60min', y='Target', data=train_df)

In [ ]:
sns.histplot(train_df.Target)